In [7]:
# Simple thermostat API using FastAPI
# importing libraries
from fastapi import FastAPI, Depends
import uvicorn
import nest_asyncio
import random
import json

app = FastAPI()


In [8]:
# Defining the thermostat class
class Thermostat:
    def __init__(self,
                 name: str,
                 temperature: int,
                 humidity: int,
                 mode: str,
                 kb_id: str,
                 ):

        self.name = name
        self.temperature = temperature
        self.humidity = humidity
        self.mode = mode
        self.kb_id = kb_id

    # Getters
    def get_name(self):
        return self.name

    def get_temperature(self):
        return self.temperature

    def get_humidity(self):
        return self.humidity

    def get_mode(self):
        return self.mode

    def get_kb_id(self):
        return self.kb_id

    def get_json(self):
        return self.__dict__

    # Setters
    def set_temperature(self, temperature: int):
        self.temperature = temperature

    def set_humidity(self, humidity: int):
        self.humidity = humidity

    def set_mode(self, mode: str):
        self.mode = mode

    def set_name(self, name: str):
        self.name = name


In [9]:
# Creating a thermostat object
thermostat = Thermostat(name="Thermostat",
                        temperature=20,
                        humidity=50,
                        mode="cool",
                        kb_id="http://127.0.0.1:8000/thermostat",
                        )

In [10]:
thermostat.get_json()

{'name': 'Thermostat',
 'temperature': 20,
 'humidity': 50,
 'mode': 'cool',
 'kb_id': 'http://127.0.0.1:8000/thermostat'}

In [11]:
# Defining the API endpoints
@app.get("/")
async def root():
    return {"message": "Welcome to the thermostat API"}

@app.get("/thermostat")
async def get_thermostat():
    return thermostat.get_json()

@app.get("/thermostat/name")
async def get_name():
    return thermostat.get_name()

@app.get("/thermostat/temperature")
async def get_temperature():
    return thermostat.get_temperature()

@app.get("/thermostat/humidity")
async def get_humidity():
    return thermostat.get_humidity()

@app.get("/thermostat/mode")
async def get_mode():
    return thermostat.get_mode()

@app.get("/thermostat/kb_id")
async def get_kb_id():
    return thermostat.get_kb_id()

@app.put("/thermostat/temperature")
async def set_temperature(temperature: int):
    thermostat.set_temperature(temperature)
    return thermostat.get_temperature()

@app.put("/thermostat/humidity")
async def set_humidity(humidity: int):
    thermostat.set_humidity(humidity)
    return thermostat.get_humidity()

@app.put("/thermostat/mode")
async def set_mode(mode: str):
    thermostat.set_mode(mode)
    return thermostat.get_mode()

@app.put("/thermostat/name")
async def set_name(name: str):
    thermostat.set_name(name)
    return thermostat.get_name()


In [13]:
# register API in the knowledge engine
import httpx

# Knowledge engine REST API URL
URL = "http://127.0.0.1:8280/rest"

# Knowledge engine REST API headers
HEADERS = {
    'Content-Type': 'application/json',
    'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
}

# Creating a smart connector
def create_smart_connector():
    # Smart connector data
    sc_data = {
        "knowledgeBaseId": "http://127.0.0.1:8000/thermostat",
        "knowledgeBaseName": "Thermostat API v0.0.1",
        "knowledgeBaseDescription": "Thermostat API v0.0.1 for the Smart Home",
        "reasonerEnabled": "false",
    }

    # Registering the smart connector
    response = httpx.post(URL + "/sc", headers=HEADERS, json=sc_data)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {HEADERS}\n')
    print(f'Smart connector data:\n {sc_data}\n')
    print(response)
    print(f'Response text: {response.text}')

create_smart_connector()

URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

Smart connector data:
 {'knowledgeBaseId': 'http://127.0.0.1:8000/thermostat', 'knowledgeBaseName': 'Thermostat API v0.0.1', 'knowledgeBaseDescription': 'Thermostat API v0.0.1 for the Smart Home', 'reasonerEnabled': 'false'}

<Response [200 OK]>
Response text: 


In [14]:
# check if the smart connector is registered
def check_smart_connector():
    headers2 = {
        #'accept': 'application/json,
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",

    }

    response = httpx.get(URL + "/sc", headers=headers2)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')

check_smart_connector()

URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [200 OK]>
Response text:
[{"knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeBaseName":"Thermostat API v0.0.1","knowledgeBaseDescription":"Thermostat API v0.0.1 for the Smart Home","reasonerEnabled":false}]


In [15]:
# register an ANSWER Knowledge Interaction with the smart connector
def register_answer_ki():
    headers2 = {
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
    }

    answer_ki_data = {
        "knowledgeInteractionType": "AnswerKnowledgeInteraction",
        "prefixes": {"rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                    "ex": "http://example.org/",
                     },

        "graphPattern":"?d rdf:type 'Device' . ?d ex:hasName 'Thermostat'. ?d ex:hasTemperature ?temperature. ?d ex:hasHumidity ?humidity. ?d ex:hasMode ?mode. ?d ex:hasName ?name."
    }

    response = httpx.post(URL + "/sc/ki", headers=headers2, json=answer_ki_data)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')

register_answer_ki()


URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [200 OK]>
Response text:
{"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/0285bc86-13d4-4835-a2b7-80f61b3d1523"}


In [16]:
# check if the Knowledge Interaction is registered
def list_all_kis():
    headers2 = {
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
    }

    response = httpx.get(URL + "/sc/ki", headers=headers2)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')

list_all_kis()


URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [404 Not Found]>
Response text:
{"messageType":"error","message":"This knowledge base has been suspended due to inactivity. Please reregister the knowledge base and its knowledge interactions."}


In [12]:
# Running the API
if __name__ == "__main__":
    nest_asyncio.apply()
    #uvicorn.run(app, host="0.0.0.0", port=8001)
    uvicorn.run(app)

INFO:     Started server process [4964]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:42736 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:42736 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:42742 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:42742 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:47088 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:47088 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4964]
